In [5]:

import json
from typing import Dict, List, Union


class GoldenDUT:
    def __init__(self):
        '''
        Initialize the 8-bit shift register to 0
        '''
        self.shift_reg = 0  # 8-bit shift register

    def load(self, clk: int, stimulus_dict: Dict[str, str]) -> Dict[str, str]:
        '''
        Update shift register and output based on inputs
        '''
        # Convert input strings to integers
        enable = int(stimulus_dict['enable'], 2)
        S = int(stimulus_dict['S'], 2)
        A = int(stimulus_dict['A'], 2)
        B = int(stimulus_dict['B'], 2)
        C = int(stimulus_dict['C'], 2)
        
        # Update shift register on rising clock edge when enabled
        if clk == 1 and enable == 1:
            # Shift left and insert S at LSB
            self.shift_reg = ((self.shift_reg << 1) | S) & 0xFF
            
        # Select output bit based on ABC
        sel = (A << 2) | (B << 1) | C  # Convert ABC to 3-bit number
        Z = (self.shift_reg >> sel) & 1  # Extract selected bit
        print((self.shift_reg),sel,Z)
        
        # Return output as binary string
        return {'Z': format(Z, 'b')}
def check_output(stimulus_list_scenario):

    dut = GoldenDUT()
    tb_outputs = []
    

    for stimulus_list in stimulus_list_scenario["input variable"]:

        
        clock_cycles = stimulus_list['clock cycles']
        clk = 0
        input_vars_list = {k: v for k, v in stimulus_list.items() if k != "clock cycles"}
        output_vars_list = {'clock cycles':clock_cycles}
        
        for i in range(clock_cycles):
            input_vars = {k:v[i] for k,v in input_vars_list.items()}
            
            output_vars = dut.load(clk,input_vars)
            for k,v in output_vars.items():
                if k not in output_vars_list:
                    output_vars_list[k] = []
                output_vars_list[k].append(v)
            clk=0 if clk==1 else 1
        
        
        tb_outputs.append(output_vars_list)

    return tb_outputs






In [6]:
stimulus_list_scenarios = [
    {
        "input variable": [
            {
                "clock cycles": 5,
                "enable": ["1", "1", "0", "1", "1"],
                "S": ["0", "1", "0", "1", "0"],
                "A": ["0", "0", "1", "1", "0"],
                "B": ["0", "1", "0", "1", "0"],
                "C": ["0", "0", "1", "0", "1"]
            }
        ]
    },
    {
        "input variable": [
            {
                "clock cycles": 3,
                "enable": ["1", "0", "1"],
                "S": ["1", "0", "1"],
                "A": ["1", "0", "1"],
                "B": ["0", "1", "0"],
                "C": ["1", "0", "1"]
            }
        ]
    }
]

outputs=[]
for stimulus_list_scenario in stimulus_list_scenarios:
    outputs.append( check_output(stimulus_list_scenario))

print(json.dumps(outputs, indent=2))

0 0 0
1 2 0
1 5 0
3 6 0
3 1 1
0 5 0
0 2 0
0 5 0
[
  [
    {
      "clock cycles": 5,
      "Z": [
        "0",
        "0",
        "0",
        "0",
        "1"
      ]
    }
  ],
  [
    {
      "clock cycles": 3,
      "Z": [
        "0",
        "0",
        "0"
      ]
    }
  ]
]
